In [913]:
import pandas as pd 
import numpy as np
# import plotly.graph_objs as go
# from plotly.offline import init_notebook_mode, iplot 

# Data augemntation and processing

This task has two data augemntation aspects: 
1) Want to know the country of each host city 
    - need a map from city name to country, use file world-cities.csv
    source: https://github.com/datasets/world-cities
2) Want to know which countries are in Europe 
    - need a map from country to continent, use file continet_map.csv
    source: https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes
    
Both files contain additional, redundant information, in case of world-cities.csv this is state/province and geolocation info. Whereas, continet_map.csv contains country names, codes, and levels of categorisation into regions and continents.


## Find the country of each host city 
Steps:
- examine and clean the world-cities.csv file
- modify raw world-cities.csv data to fit requirements 
- merge with the master olympic dataset

In [986]:
# Load Cleaned Olympic data 
clean = pd.read_csv('all_data/clean_data.csv')

In [987]:
# Load city data 
city = pd.read_csv('all_data/world-cities.csv')

In [988]:
city.head()

,name,country,subcountry,geonameid
0,les Escaldes,Andorra,Escaldes-Engordany,3040051
1,Andorra la Vella,Andorra,Andorra la Vella,3041563
2,Umm al Qaywayn,United Arab Emirates,Umm al Qaywayn,290594
3,Ras al-Khaimah,United Arab Emirates,Raʼs al Khaymah,291074
4,Khawr Fakkān,United Arab Emirates,Ash Shāriqah,291696


In [989]:
city.isnull().sum() # name and country have no nan values, thats ok 

name           0
country        0
subcountry    26
geonameid      0
dtype: int64

In [990]:
clean_uniq = clean.City.unique()

In [991]:
city_uniq = city.name.unique()

In [992]:
# check if city_uniq contains all elements in merged_uniq
all(elem in city_uniq for elem in clean_uniq)

False

In [993]:
for elem in merged_uniq: 
    if elem not in city_uniq:
        print(elem)

Lake Placid
Torino
Athina
Squaw Valley
Cortina d'Ampezzo
Roma
Montreal
Moskva
Chamonix
Sankt Moritz


Some City names in the Olympic data set are not in english are wrong or do not appear in the world_cities file. Thankfully, this is a small amount of cities and ca be manually cleaned. 

The file world_cities does not contain ski resorts. My initial idea was to overwrite this ski resort names with nearby city names. However, this is a loss of information. Furthermore, I run into problems with this approach, where for instance in France I was trying to replace Chamonix (not in world_cities list) with the nearby town of Albertville, only to find out that Albertville hosted a separate winter Olympics. Therefore, in order to keep true to the information in the original dataset, I added new city-country mappings to the world_cities data frame instead. 

In [994]:
# rename miss-spelled city names, which appear in the world_cities df
err = {'Torino': 'Turin','Athina':'Athens','Roma':'Rome', 'Montreal':'Montréal','Moskva':'Moscow'}

for key in err:
    clean.loc[clean.City == key, 'City']= err[key]

In [995]:
# create additional city to country mapping
dd = {'Lake Placid' : ['USA', 'New_York',0],
     'Squaw Valley': ['USA', 'California',0],
     'Chamonix':['France','nan', 0],
     'Sankt Moritz':['Switzerland','nan',0]}

In [996]:
df = pd.DataFrame.from_dict(dd, orient='index')
df.rename(columns={ 0:'country',1:'subcountry',2:'geonameid'},inplace=True)
df.reset_index(inplace=True)
df.rename(columns={'index':'name'},inplace=True)

In [997]:
df

,name,country,subcountry,geonameid
0,Chamonix,France,nan,0
1,Sankt Moritz,Switzerland,nan,0
2,Lake Placid,USA,New_York,0
3,Squaw Valley,USA,California,0


In [998]:
# add new mapping to the city df
city = city.append(df, ignore_index=True)

In [999]:
city.tail() # verify worked 

,name,country,subcountry,geonameid
23017,Chitungwiza,Zimbabwe,Harare,1106542
23018,Chamonix,France,nan,0
23019,Sankt Moritz,Switzerland,nan,0
23020,Lake Placid,USA,New_York,0
23021,Squaw Valley,USA,California,0


In [1000]:
# clean up city df to keep only relevant info and prepare for merge
city.drop(['subcountry','geonameid'],axis=1,inplace=True)
city.rename(columns={'name':'City','country':'Host_Country'},inplace=True)

# rename countries to match the olympics data frame format
city.loc[city.Host_Country == 'United Kingdom','Host_Country'] = 'UK'
city.loc[city.Host_Country == 'United States','Host_Country'] = 'USA'

In [1001]:
city[city.City=='Athens']

,City,Host_Country
7976,Athens,Greece
20022,Athens,USA
20465,Athens,USA


Duh... of course many cities have duplicate names, i.e. they exist in more than one coutry.... so I have to clean the data some more... <br>
__In hindsight I would have been better off just making this mapping for the 42 host cities from scratch __

In [1002]:
city.loc[city.City == 'London','Host_Country']= 'UK'
city.loc[city.City == 'Barcelona','Host_Country']= 'Spain'
city.loc[city.City == 'Sydney','Host_Country']= 'Australia'
city.loc[city.City == 'Athens','Host_Country']= 'Greece'
city.loc[city.City == 'Melbourne','Host_Country']= 'Australia'
city.loc[city.City == 'Rome','Host_Country']= 'Italy'
city.loc[city.City == 'Amsterdam','Host_Country']= 'Netherlands'
city.loc[city.City == 'Moscow','Host_Country']= 'Moscow'
city.loc[city.City == 'Vancouver','Host_Country']= 'Canada'

In [1003]:
city.drop_duplicates(inplace=True)

In [1004]:
# MERGE
master = pd.merge(clean, city ,on='City',how='left')

In [1005]:
# sanity check 
assert (clean.shape[0] == master.shape[0]) # no new rows created 

In [1006]:
master[master.City =='Barcelona']['Host_Country'].unique()

array(['Spain'], dtype=object)

In [1007]:
master[master.City =='Athens']['Host_Country'].unique()

array(['Greece'], dtype=object)

In [1008]:
master[master.City =='Sankt Moritz']['Host_Country'].unique()

array(['Switzerland'], dtype=object)

## Find the continent of each host city and nation 
Steps:
- examine and clean the continet_map.csv
- modify raw continet_map.csv data to fit requirements
    - aside: take a closer look at how data from European countries which ceased to exist is treated 
- merge with the master olympic dataset, creatine two new columns; for the continet of the hoast country and the continent of participating countries 

In [1009]:
cont = pd.read_csv('all_data/continet_map.csv')

In [1010]:
# sanity check
cont[cont.region=="Europe"].count() 

name                        51
alpha-2                     51
alpha-3                     51
country-code                51
iso_3166-2                  51
region                      51
sub-region                  51
intermediate-region          2
region-code                 51
sub-region-code             51
intermediate-region-code     2
dtype: int64

In [1011]:
cont[cont.region=="Europe"]

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
5,Andorra,AD,AND,20,ISO 3166-2:AD,Europe,Southern Europe,NaN,150.0,39.0,NaN
14,Austria,AT,AUT,40,ISO 3166-2:AT,Europe,Western Europe,NaN,150.0,155.0,NaN
20,Belarus,BY,BLR,112,ISO 3166-2:BY,Europe,Eastern Europe,NaN,150.0,151.0,NaN
21,Belgium,BE,BEL,56,ISO 3166-2:BE,Europe,Western Europe,NaN,150.0,155.0,NaN
28,Bosnia and Herzegovina,BA,BIH,70,ISO 3166-2:BA,Europe,Southern Europe,NaN,150.0,39.0,NaN
34,Bulgaria,BG,BGR,100,ISO 3166-2:BG,Europe,Eastern Europe,NaN,150.0,151.0,NaN
55,Croatia,HR,HRV,191,ISO 3166-2:HR,Europe,Southern Europe,NaN,150.0,39.0,NaN
59,Czechia,CZ,CZE,203,ISO 3166-2:CZ,Europe,Eastern Europe,NaN,150.0,151.0,NaN


### New data sanity check:
- Yes there are aprox 50 countries in Europe, intermediate-regions are UK channel islands, some regions (i.e Giblartar) will not have their own olympic representation. Therefore, ovreall there are more European countries,regions in this dataset, than those with Olympic teams. 
- From visula inspection of df, where the region == Europe, values match reality.<br>

### Actions:
Use the 'region', 'sub-region' and 'Country' columns from the new data frame, drop the remaining information. Then merge the 'region' and 'sub-region' columns to the olympic data frame using 'Country' as the key. Take care when merging data frames to include only the european countries which had Olympic teams.

In [1012]:
cont.rename(columns={'name':'Country'},inplace=True) # rename column to match the main data naming convention

In [1013]:
cont.drop(['alpha-2', 'alpha-3', 'country-code', 'iso_3166-2','intermediate-region', 'region-code', 'sub-region-code',
       'intermediate-region-code'],axis=1,inplace=True)

In [1014]:
# what are the unique regions available? which are NAN?
cont.region.unique()

array(['Asia', 'Europe', 'Africa', 'Oceania', 'Americas', nan],
      dtype=object)

In [1015]:
cont.isnull().sum() 

Country       0
region        1
sub-region    1
dtype: int64

In [1016]:
cont[cont.Country=='Antarctica']  # Antarctica is NaN thats ok, not important here

,Country,region,sub-region
8,Antarctica,NaN,NaN


In [1017]:
cont[cont.region=='Europe'].tail()

,Country,region,sub-region
213,Svalbard and Jan Mayen,Europe,Northern Europe
214,Sweden,Europe,Northern Europe
215,Switzerland,Europe,Western Europe
232,Ukraine,Europe,Eastern Europe
234,United Kingdom of Great Britain and Northern I...,Europe,Northern Europe


In [1018]:
assert(clean[clean.Country == 'Macedonia'].count().sum()) # check Macedonia in main df... 

In [1019]:
cont[cont.Country == 'Cyprus']

,Country,region,sub-region
58,Cyprus,Asia,Western Asia


Inspecting the above table (expanded) see a few problems with country names: 
        - UK is 'United Kingdom of Great Britain and Northern Ireland',
        - Moldova is 'Moldova (Republic of)', 
        - Macedonia is 'Macedonia (the former Yugoslav Republic of)', 
        - Russia is 'Russian Federation'. 
        - Czechia is the Czech Republic... 
Aditionally:
        - Cyprus is not in Asia... 
        - Kosovo is in Europe...
        - rename 'United States of America' to USA to match mian data frame
        
These have to be renamed before merge, since the countries take part in the Olympics and exist in the main data frame.
<br>

__Note:__ I will follow the definition of Europe from notebook 2: only countries with their territory fully within the European continent will be considered. Thus, need to drop Russia to be in line with definition, so I will assign Russia to the 'Asia' region in the cells below. 

In [1020]:
# change relevant country names to match main DF names
cont.loc[cont.Country == 'Moldova (Republic of)',['Country','sub-region']]= 'Moldova', 'Southern Europe'
cont.loc[cont.Country == 'Macedonia (the former Yugoslav Republic of)',\
         ['Country','sub-region']]= 'Macedonia', 'Southern Europe'
cont.loc[cont.Country == 'United Kingdom of Great Britain and Northern Ireland',\
         ['Country','sub-region']]= 'UK', 'Northern Europe'
cont.loc[cont.Country == 'Czechia',['Country','region','sub-region']]= 'Czech Republic','Europe', 'Eastern Europe'
cont.loc[cont.Country == 'Cyprus',['region','sub-region']]=  'Europe','Southern Europe'
cont.loc[cont.Country == 'Russian Federation',['Country','region','sub-region']]= 'Russia', 'Asia', 'Asia'
cont.loc[cont.Country == 'United States of America','Country']= 'USA'


In [1021]:
# cosovo not in cont DF

In [1022]:
cont.loc[cont.Country == 'Kosovo',['region','sub-region']]= 'Europe','Southern Europe'

In [1023]:
cont.loc[-1] = ['Kosovo','Europe','Southern Europe']

In [1024]:
cont.tail(2)

,Country,region,sub-region
248,Zimbabwe,Africa,Sub-Saharan Africa
-1,Kosovo,Europe,Southern Europe


#### Delve deeper into European data/ history
Prompted by the data errors encountered around Czech Republic and Kosovo. I decided to have a closer look at how 'Former Members' of the European Olympic Committee are treated in the dataset. This are:
<br>
- Czechoslovakia, 
    - Prior to 1993 the Czech Republic and Slovakia were one country. Today, using kaggle data, __all the medals won by Czechoslovakia are attributed to the Czech Republic__ and none to Slovakia. This is a possible mistake. 
- East Germany
    - This was the part of Germany belonging to the Eastern block during the Cold War. This medals are unambiguously attributed to Germany.
- Soviet Union
   - This is USSR, medals are unambiguously attributed to Russia.
- Yugoslavia  
   - The Socialist Federal Republic of Yugoslavia was a country founded in the aftermath of World War II. Like other countries under Soviet influence, its dissolution occurred shortly after the end of the Cold War, in 1992. Yugoslavia comprised six socialist republics: Bosnia and Herzegovina, Croatia, Macedonia, Montenegro, Serbia, and Slovenia. In addition, it included the autonomous province of Kosovo. 
   __In the kaggle dataset all Yugoslavian medals and athletes are attributed to Serbia!__  This is also a possible mistake.
- Serbia and Montenegro
    - Country formed after the collapse of Yugoslavia, comprising some of its former states: Serbia, Montenegro, Kosovo. This existed until Montenegro (2006) and Kosovo (2008) progressively separated from Serbia also declaring independence. However, again, all medals won during this period in time are attributed to Serbia. 

In summary, when it comes to ex-Yugoslavian countries, such as Croatia, Slovenia etc. They have all been assigned new NOC codes upon declaring independence. Since then, they figure in the kaggle dataset as independent entities. Prior to this time, all Yugoslavian medals are attributed to Serbia, which can be misleading and skew results in favour of this country at the same time penalising the 6 other ex-Yugoslavian nations. 

__Action:__ Decide to do nothing, cleaning this dataset to re-distribute athletes and medals among ex-Yugoslavian countries is beyond the scope of this work. However, it is important to be aware of this data 'problem' especially when looking closely at Europe in the Olympics; care should be taken when interpreting future results.

Czechoslovakia

In [1025]:
clean[clean.NOC == 'TCH']['Team'].unique()

array(['Czechoslovakia', 'Czechoslovakia-3', 'Czechoslovakia-1',
       'Czechoslovakia-2'], dtype=object)

In [1026]:
np.sort(clean[clean.NOC == 'TCH']['Year'].unique())

array([1920, 1924, 1928, 1932, 1936, 1948, 1952, 1956, 1960, 1964, 1968,
       1972, 1976, 1980, 1984, 1988, 1992])

In [1027]:
np.sort(clean[clean.NOC == 'CZE']['Year'].unique())

array([1994, 1996, 1998, 2000, 2002, 2004, 2006, 2008, 2010, 2012, 2014,
       2016])

In [1028]:
np.sort(clean[clean.NOC == 'SVK']['Year'].unique())

array([1994, 1996, 1998, 2000, 2002, 2004, 2006, 2008, 2010, 2012, 2014,
       2016])

As can be seen from above cells, the NOC code TCH corresponding to the Czech Republic actually contains 'team' Czechoslovakia.   

Yugoslavia

In [1029]:
np.sort(clean[clean.NOC == 'YUG']['Year'].unique())

array([1920, 1924, 1928, 1932, 1936, 1948, 1952, 1956, 1960, 1964, 1968,
       1972, 1976, 1980, 1984, 1988, 1992])

In [1030]:
clean[clean.NOC == 'YUG']['Country'].unique()

array(['Serbia'], dtype=object)

Serbia and Montenegro

In [1031]:
np.sort(clean[clean.NOC == 'SCG']['Year'].unique())

array([1996, 1998, 2000, 2002, 2004, 2006])

In [1032]:
clean[clean.NOC == 'SCG']['Country'].unique()

array(['Serbia'], dtype=object)

#### Merge and select European countries 

In [1033]:
# further clean up cont DF to have intended column names
cont.rename(columns={'region':'Continent','sub-region':'sub_Continent'},inplace=True)
cont.head()

,Country,Continent,sub_Continent
0,Afghanistan,Asia,Southern Asia
1,Åland Islands,Europe,Northern Europe
2,Albania,Europe,Southern Europe
3,Algeria,Africa,Northern Africa
4,American Samoa,Oceania,Polynesia


In [1034]:
# 1st merge on 'Country' to get the continent of each participating nation
merged = pd.merge(master,cont,on='Country',how='left')

In [1035]:
# rename agin to have intended column names
cont.rename(columns={'Country':'Host_Country','Continent':'Host_Continent','sub_Continent':'Host_sub_Continent'},inplace=True)

In [1036]:
# 2nd merge on 'Host_Country' to get the continent of each host nation
final = pd.merge(merged,cont,on='Host_Country',how='left')

In [1037]:
final.drop('Unnamed: 0',axis=1,inplace=True)

### Final Sanity Checks 

In [1038]:
# make a smaller DF with only Europe included 
europe = final[final.Continent=="Europe"]

In [1039]:
np.sort(europe['Country'].unique())

array(['Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium',
       'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy',
       'Kosovo', 'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg',
       'Macedonia', 'Malta', 'Moldova', 'Monaco', 'Montenegro',
       'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania',
       'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
       'Switzerland', 'UK', 'Ukraine'], dtype=object)

In [1040]:
print('Number of countries classed as "European" that have an Olympic team is:', len(europe['Country'].unique()))

Number of countries classed as "European" that have an Olympic team is: 44


#### Explore which countries constitute european sub-regons classes
- generally all looks ok 

In [1041]:
europe['sub_Continent'].unique()

array(['Northern Europe', 'Western Europe', 'Eastern Europe',
       'Southern Europe'], dtype=object)

In [1042]:
# Explore what constitutes eastern europe - note Russia is excluded
europe[europe['sub_Continent'] == 'Eastern Europe']['Country'].unique()

array(['Romania', 'Bulgaria', 'Belarus', 'Hungary', 'Ukraine', 'Poland',
       'Czech Republic', 'Slovakia'], dtype=object)

In [1043]:
europe[europe['sub_Continent'] == 'Southern Europe']['Country'].unique()

array(['Spain', 'Italy', 'Greece', 'Slovenia', 'Croatia', 'Portugal',
       'Malta', 'Cyprus', 'Serbia', 'Moldova', 'Bosnia and Herzegovina',
       'San Marino', 'Andorra', 'Macedonia', 'Montenegro', 'Albania',
       'Kosovo'], dtype=object)

In [1044]:
europe[europe['sub_Continent'] == 'Northern Europe']['Country'].unique()

array(['Denmark', 'Finland', 'Norway', 'Estonia', 'Ireland', 'Sweden',
       'UK', 'Latvia', 'Lithuania', 'Iceland'], dtype=object)

In [1045]:
europe[europe['sub_Continent'] == 'Western Europe']['Country'].unique()

array(['Netherlands', 'France', 'Germany', 'Belgium', 'Switzerland',
       'Monaco', 'Luxembourg', 'Austria', 'Liechtenstein'], dtype=object)

### Save European data

In [ ]:
# Save European data
# europe.to_csv('all_data/europe_data.csv')

In [1049]:
# Save world data with added columns
final.to_csv('all_data/extended_data.csv')

### CONCLUDE
Instead of trying to 'cut corners' and sue pre-existing datasets it would have been much easier and less time consuming to manually select the countries which constitute Europe and map host cities to hist countries... 